In [2]:
import pandas as pd
import mysql.connector
from mysql.connector import Error

In [3]:
# Fonction pour établir la connexion à la base de données
def create_connection(host, user, password, database):
    try:
        connection = mysql.connector.connect(
            host=host,
            user=user,
            password=password,
            database=database
        )
        if connection.is_connected():
            print(f"Connected to MySQL Server: {host} with database: {database}")
            return connection
    except Error as e:
        print(f"Error: {e}")
        return None

In [4]:
# Fonction pour exécuter une requête SQL
def execute_query(connection, query):
    try:
        cursor = connection.cursor()
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"Error: {e}")

In [5]:
# Création de la base de données et des tables
def create_database_tables(connection):
    create_database_query = "CREATE DATABASE IF NOT EXISTS films_database"
    execute_query(connection, create_database_query)

    use_database_query = "USE films_database"
    execute_query(connection, use_database_query)

    create_movies_table_query = """
    CREATE TABLE IF NOT EXISTS movies (
        id INT AUTO_INCREMENT PRIMARY KEY,
        title VARCHAR(255) NOT NULL,
        release_year INT,
        genre VARCHAR(255),
        director VARCHAR(255),
        imdb_rating FLOAT,
        keywords VARCHAR(255),
        country VARCHAR(255),
        revenue FLOAT
    )
    """
    execute_query(connection, create_movies_table_query)

    create_actors_table_query = """
    CREATE TABLE IF NOT EXISTS actors (
        id INT AUTO_INCREMENT PRIMARY KEY,
        name VARCHAR(255) NOT NULL,
        movie_id INT,
        FOREIGN KEY (movie_id) REFERENCES movies(id)
    )
    """
    execute_query(connection, create_actors_table_query)

In [6]:
# Chargement des données depuis un fichier CSV dans la base de données
def load_data_into_tables(connection, movies_data):
    for _, row in movies_data.iterrows():
        movie_values = [row["title"], row["release_year"], row["genre"], row["director"], row["imdb_rating"], row["keywords"], row["country"], row["revenue"]]
        create_record(connection, "movies", movie_values)
        movie_id = get_last_inserted_id(connection)

        actors = row["actors"].split(", ")
        for actor in actors:
            create_record(connection, "actors", [actor, movie_id])


In [7]:
# Fonction pour obtenir le dernier ID inséré dans une table
def get_last_inserted_id(connection):
    cursor = connection.cursor()
    cursor.execute("SELECT LAST_INSERT_ID()")
    last_inserted_id = cursor.fetchone()[0]
    return last_inserted_id

In [8]:
# Fonction pour créer un enregistrement dans une table
def create_record(connection, table_name, values):
    values_str = ', '.join([f"'{str(value)}'" for value in values])
    insert_query = f"INSERT INTO {table_name} VALUES (NULL, {values_str})"
    execute_query(connection, insert_query)

In [9]:
# Fonction pour exécuter une opération de lecture (Read)
def read_records(connection, query):
    cursor = connection.cursor(dictionary=True)
    cursor.execute(query)
    records = cursor.fetchall()
    return records

In [10]:
# Paramètres de connexion à la base de données
host = "localhost"
user = "root"
password = "toor"

In [11]:
# Connexion à la base de données
connection = create_connection(host, user, password, None)

Connected to MySQL Server: localhost with database: None


In [12]:
# Création de la base de données et des tables
create_database_tables(connection)

Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully


In [13]:
# Utilisation de la base de données
connection = create_connection(host, user, password, "films_database")

Connected to MySQL Server: localhost with database: films_database


In [14]:
# Charger les données depuis le fichier CSV
try:
    movies_data = pd.read_csv('movies_imdb.csv', encoding='utf-8', sep=',')
except pd.errors.ParserError:
    print("Une erreur s'est produite lors de la lecture du fichier CSV.")
    # Gérer l'erreur ici, comme ignorer la ligne ou afficher un message d'erreur

In [16]:
# Requêtes SQL pour répondre aux demandes
# Exemple : Top 10 des films les plus rentables
query_top_10_profitable_movies = """
SELECT title, revenue
FROM movies
ORDER BY revenue DESC
LIMIT 10
"""
top_10_profitable_movies = read_records(connection, query_top_10_profitable_movies)
print("Top 10 des films les plus rentables :")
for movie in top_10_profitable_movies:
    print(movie["title"], "-", movie["revenue"])

Top 10 des films les plus rentables :
